# Registro Retributivo - Análisis de Datos

Este notebook se utiliza para cargar y analizar los datos del archivo "PRUEBA QLICK JULIO 2025.xlsx".

In [69]:
# IMPORTAR LIBRERÍAS NECESARIAS
import pandas as pd
import numpy as np
import os
import warnings
from IPython.display import display

# Suprimir warnings innecesarios
warnings.filterwarnings('ignore')

print("✓ Librerías importadas exitosamente")

✓ Librerías importadas exitosamente


In [70]:
# ============================================================
# CONFIGURACIÓN GENERAL
# ============================================================

# NOTA: Modifica esta sección según tu archivo Excel específico

# 1. RUTA DEL ARCHIVO EXCEL
# Coloca aquí la ruta completa a tu archivo Excel
archivo_excel = r"C:\Users\juanf\Downloads\Equiality Momentum\datos\PRUEBA QLICK JULIO 2025.xlsx"

# 2. NOMBRE DE LA HOJA PRINCIPAL
# Especifica el nombre de la hoja que contiene los datos principales
# Si no se especifica, se utilizará la primera hoja disponible
hoja_principal = "BASE GENERAL"  # Cambiar por el nombre de tu hoja

# 3. CONFIGURACIÓN DE COLUMNAS
# Especifica si se debe eliminar la primera columna (típicamente índices de Excel)
eliminar_primera_columna = True

# 4. MODO DEBUG
# Controla la cantidad de información mostrada durante el procesamiento
DEBUG_MODE = False  # Cambiar a True para ver información detallada de debug

# ============================================================
# CARGA Y VALIDACIÓN DEL ARCHIVO
# ============================================================

# Verificar que el archivo existe
if os.path.exists(archivo_excel):
    print(f"✓ Archivo encontrado: {archivo_excel}")
    
    # Cargar el archivo Excel
    try:
        # Primero veamos qué hojas tiene el archivo
        excel_file = pd.ExcelFile(archivo_excel)
        print(f"\nHojas disponibles en el archivo:")
        for i, sheet in enumerate(excel_file.sheet_names, 1):
            print(f"  {i}. {sheet}")
        
        # Determinar qué hoja cargar
        hoja_a_cargar = None
        if hoja_principal and hoja_principal in excel_file.sheet_names:
            hoja_a_cargar = hoja_principal
        elif excel_file.sheet_names:
            hoja_a_cargar = excel_file.sheet_names[0]  # Primera hoja como fallback
            print(f"⚠️  Hoja '{hoja_principal}' no encontrada, usando '{hoja_a_cargar}'")
        
        if hoja_a_cargar:
            df = pd.read_excel(archivo_excel, sheet_name=hoja_a_cargar)
            print(f"\n✓ Datos cargados exitosamente desde la hoja: '{hoja_a_cargar}'")
            print(f"  - Dimensiones originales: {df.shape[0]} filas x {df.shape[1]} columnas")
            
            # Eliminar la primera columna si está configurado
            if eliminar_primera_columna and len(df.columns) > 0:
                primera_columna = df.columns[0]
                df = df.drop(columns=[primera_columna])
                print(f"  - Primera columna eliminada: '{primera_columna}'")
                print(f"  - Dimensiones finales: {df.shape[0]} filas x {df.shape[1]} columnas")
            else:
                print("  - No se eliminó la primera columna")
        else:
            raise Exception("No se pudo determinar qué hoja cargar")
            
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        
else:
    print(f"❌ Archivo no encontrado: {archivo_excel}")
    print("Por favor, verifica la ruta del archivo en la sección CONFIGURACIÓN GENERAL.")

✓ Archivo encontrado: C:\Users\juanf\Downloads\Equiality Momentum\datos\PRUEBA QLICK JULIO 2025.xlsx

Hojas disponibles en el archivo:
  1. PRESENTACIÓN
  2. BASE GENERAL
  3. SB equiparados1
  4.  CONTROL QLICK 1
  5. COMPLEMENTOS SALARIALES
  6. COMPLEMENTOS EXTRASALARIALES
  7. SB equiparados 2
  8.  CONTROL QLICK 2
  9. CONTROL QLICK 3
  10.  CONTROL QLICK 4
  11.  CONTROL QLICK 5.1
  12. CONTROL QLICK 5.2
  13. CESES
  14. PERMISOS CONCILIACIÓN
  15. EXCEDENCIAS
  16. SUSPENSIONES
  17. TURNOS
  18. Datos de formación del PIO

✓ Datos cargados exitosamente desde la hoja: 'BASE GENERAL'
  - Dimensiones originales: 150 filas x 267 columnas
  - Primera columna eliminada: 'Unnamed: 0'
  - Dimensiones finales: 150 filas x 266 columnas

✓ Datos cargados exitosamente desde la hoja: 'BASE GENERAL'
  - Dimensiones originales: 150 filas x 267 columnas
  - Primera columna eliminada: 'Unnamed: 0'
  - Dimensiones finales: 150 filas x 266 columnas


## Cálculo de Datos Equiparados

Los datos equiparados se calculan para determinar cuánto habría cobrado la persona si hubiera estado trabajando bajo una misma situación contractual durante los 12 meses y a jornada completa.

### Fórmulas aplicadas:

**Para salarios base:**
- Salario base equiparado = [Salario base anual efectivo] × (1/[Coef. TP]) × (12/[Meses Trabajados])

**Para complementos:**
- Si es normalizable y/o anualizable: se usa el valor equiparado correspondiente
- Si NO es ni normalizable ni anualizable: se usa el valor efectivo original (no se procesa)
- Los totales de complementos equiparados suman correctamente: valor equiparado para procesables, valor efectivo para no procesables

In [71]:
# CONFIGURACIÓN MANUAL DE COLUMNAS
# Usando los nombres exactos del Excel

# Mapeo con los nombres exactos de las columnas
mapeo_columnas = {
    'meses_trabajados': '¿Cuántos meses ha trabajado?',
    'coef_tp': '% de jornada',
    'salario_base_efectivo': 'Salario base anual efectivo',
    'complementos_salariales_efectivo': 'Compltos Salariales efectivo',
    'complementos_extrasalariales_efectivo': 'Compltos Extrasalariales efectivo'
}

print("=== VERIFICACIÓN DE COLUMNAS CON NOMBRES EXACTOS ===")

# Verificar que las columnas existen en el DataFrame
columnas_verificadas = {}
for clave, nombre_col in mapeo_columnas.items():
    if nombre_col in df.columns:
        columnas_verificadas[clave] = nombre_col
        print(f"✓ {clave}: {nombre_col}")
        
        # Mostrar información adicional sobre la columna
        valores_no_nulos = df[nombre_col].dropna()
        if len(valores_no_nulos) > 0:
            print(f"    Registros con datos: {len(valores_no_nulos)}/{len(df)}")
            if nombre_col == '% de jornada':
                print(f"    Valores únicos: {sorted(df[nombre_col].dropna().unique())}")
        else:
            print(f"    ⚠️  Sin datos en esta columna")
    else:
        print(f"❌ {clave}: COLUMNA '{nombre_col}' NO ENCONTRADA")
        
        # Buscar nombres similares
        columnas_similares = [col for col in df.columns if any(palabra.lower() in col.lower() 
                             for palabra in nombre_col.split())]
        if columnas_similares:
            print(f"    Columnas similares encontradas: {columnas_similares}")

print(f"\n=== RESUMEN FINAL ===")
print(f"Columnas configuradas correctamente: {len(columnas_verificadas)}/{len(mapeo_columnas)}")

# Crear df_equiparado como copia del DataFrame original
df_equiparado = df.copy()
print(f"✓ df_equiparado creado: {df_equiparado.shape[0]} filas x {df_equiparado.shape[1]} columnas")

=== VERIFICACIÓN DE COLUMNAS CON NOMBRES EXACTOS ===
✓ meses_trabajados: ¿Cuántos meses ha trabajado?
    Registros con datos: 150/150
✓ coef_tp: % de jornada
    Registros con datos: 150/150
    Valores únicos: [np.float64(0.33), np.float64(0.5), np.float64(0.75), np.float64(0.758), np.float64(0.7802), np.float64(0.875), np.float64(1.0)]
✓ salario_base_efectivo: Salario base anual efectivo
    Registros con datos: 145/150
✓ complementos_salariales_efectivo: Compltos Salariales efectivo
    Registros con datos: 150/150
✓ complementos_extrasalariales_efectivo: Compltos Extrasalariales efectivo
    Registros con datos: 41/150

=== RESUMEN FINAL ===
Columnas configuradas correctamente: 5/5
✓ df_equiparado creado: 150 filas x 266 columnas


In [72]:
# CARGAR CONFIGURACIÓN DE COMPLEMENTOS
print("=== CARGANDO CONFIGURACIÓN DE COMPLEMENTOS ===")

configuracion_complementos = {}

# Nombres exactos de las columnas que buscamos
nombres_columnas_config = {
    'codigo': 'Cod',
    'normalizable': '¿Es Normalizable?',
    'anualizable': '¿Es Anualizable?'
}

def is_positive_response(value):
    """Check if a value represents a positive response (Sí/Si/YES)"""
    if pd.isna(value):
        return False
    
    # Normalize the string: strip whitespace and convert to lowercase
    normalized = str(value).strip().lower()
    
    # Check for both accented and unaccented versions
    return normalized in ['sí', 'si', 'yes', 'y', '1', 'true']

try:
    # Cargar configuración de complementos salariales
    if 'COMPLEMENTOS SALARIALES' in excel_file.sheet_names:
        print("Cargando configuración de COMPLEMENTOS SALARIALES...")
        df_comp_sal_config = pd.read_excel(archivo_excel, sheet_name='COMPLEMENTOS SALARIALES')
        
        if DEBUG_MODE:
            print(f"Columnas encontradas: {list(df_comp_sal_config.columns)}")
            print(f"Primeras filas:")
            display(df_comp_sal_config.head())
        
        # Verificar que existen las columnas necesarias
        columnas_requeridas = {}
        for clave, nombre_col in nombres_columnas_config.items():
            if nombre_col in df_comp_sal_config.columns:
                columnas_requeridas[clave] = nombre_col
                if DEBUG_MODE:
                    print(f"✓ Columna '{nombre_col}' encontrada")
            else:
                print(f"❌ Columna '{nombre_col}' NO encontrada")
                if DEBUG_MODE:
                    print(f"   Columnas disponibles: {list(df_comp_sal_config.columns)}")
        
        # Procesar la configuración de complementos salariales si tenemos todas las columnas
        if len(columnas_requeridas) == len(nombres_columnas_config):
            for _, row in df_comp_sal_config.iterrows():
                codigo_val = row[columnas_requeridas['codigo']]
                if pd.notna(codigo_val):  # Si hay código
                    codigo = str(codigo_val).strip()
                    
                    # Procesar normalizable usando la función mejorada
                    normalizable_val = row[columnas_requeridas['normalizable']]
                    es_normalizable = is_positive_response(normalizable_val)
                    
                    # Procesar anualizable usando la función mejorada
                    anualizable_val = row[columnas_requeridas['anualizable']]
                    es_anualizable = is_positive_response(anualizable_val)
                    
                    configuracion_complementos[codigo] = {
                        'tipo': 'salarial',
                        'es_normalizable': es_normalizable,
                        'es_anualizable': es_anualizable
                    }
            
            print(f"✓ Configuración de complementos salariales cargada: {len([k for k, v in configuracion_complementos.items() if v['tipo'] == 'salarial'])} códigos")
        else:
            print("❌ No se pudieron encontrar todas las columnas necesarias en COMPLEMENTOS SALARIALES")
    else:
        print("❌ Hoja 'COMPLEMENTOS SALARIALES' no encontrada")
    
    # Cargar configuración de complementos extrasalariales
    if 'COMPLEMENTOS EXTRASALARIALES' in excel_file.sheet_names:
        print("\nCargando configuración de COMPLEMENTOS EXTRASALARIALES...")
        df_comp_extra_config = pd.read_excel(archivo_excel, sheet_name='COMPLEMENTOS EXTRASALARIALES')
        
        if DEBUG_MODE:
            print(f"Columnas encontradas: {list(df_comp_extra_config.columns)}")
            print(f"Primeras filas:")
            display(df_comp_extra_config.head())
        
        # Verificar que existen las columnas necesarias
        columnas_requeridas = {}
        for clave, nombre_col in nombres_columnas_config.items():
            if nombre_col in df_comp_extra_config.columns:
                columnas_requeridas[clave] = nombre_col
                if DEBUG_MODE:
                    print(f"✓ Columna '{nombre_col}' encontrada")
            else:
                print(f"❌ Columna '{nombre_col}' NO encontrada")
                if DEBUG_MODE:
                    print(f"   Columnas disponibles: {list(df_comp_extra_config.columns)}")
        
        # Procesar la configuración de complementos extrasalariales si tenemos todas las columnas
        if len(columnas_requeridas) == len(nombres_columnas_config):
            for _, row in df_comp_extra_config.iterrows():
                codigo_val = row[columnas_requeridas['codigo']]
                if pd.notna(codigo_val):  # Si hay código
                    codigo = str(codigo_val).strip()
                    
                    # Procesar normalizable usando la función mejorada
                    normalizable_val = row[columnas_requeridas['normalizable']]
                    es_normalizable = is_positive_response(normalizable_val)
                    
                    # Procesar anualizable usando la función mejorada
                    anualizable_val = row[columnas_requeridas['anualizable']]
                    es_anualizable = is_positive_response(anualizable_val)
                    
                    configuracion_complementos[codigo] = {
                        'tipo': 'extrasalarial',
                        'es_normalizable': es_normalizable,
                        'es_anualizable': es_anualizable
                    }
            
            print(f"✓ Configuración de complementos extrasalariales cargada: {len([k for k, v in configuracion_complementos.items() if v['tipo'] == 'extrasalarial'])} códigos")
        else:
            print("❌ No se pudieron encontrar todas las columnas necesarias en COMPLEMENTOS EXTRASALARIALES")
    else:
        print("❌ Hoja 'COMPLEMENTOS EXTRASALARIALES' no encontrada")
    
    # Mostrar resumen de la configuración cargada
    print(f"\n=== RESUMEN DE CONFIGURACIÓN DE COMPLEMENTOS ===")
    print(f"Total de códigos configurados: {len(configuracion_complementos)}")
    
    if configuracion_complementos and DEBUG_MODE:
        print(f"\nConfiguración por código:")
        for codigo, config in sorted(configuracion_complementos.items()):
            normalizable = "Sí" if config['es_normalizable'] else "No"
            anualizable = "Sí" if config['es_anualizable'] else "No"
            print(f"  {codigo} ({config['tipo']}): Normalizable={normalizable}, Anualizable={anualizable}")
    
    # Estadísticas siempre visibles
    if configuracion_complementos:
        total_salariales = len([k for k, v in configuracion_complementos.items() if v['tipo'] == 'salarial'])
        total_extrasalariales = len([k for k, v in configuracion_complementos.items() if v['tipo'] == 'extrasalarial'])
        total_normalizables = len([k for k, v in configuracion_complementos.items() if v['es_normalizable']])
        total_anualizables = len([k for k, v in configuracion_complementos.items() if v['es_anualizable']])
        
        print(f"\nEstadísticas:")
        print(f"  - Complementos salariales: {total_salariales}")
        print(f"  - Complementos extrasalariales: {total_extrasalariales}")
        print(f"  - Normalizables: {total_normalizables}")
        print(f"  - Anualizables: {total_anualizables}")

    
except Exception as e:
    print(f"❌ Error al cargar configuración de complementos: {e}")
    if DEBUG_MODE:
        import traceback
        traceback.print_exc()
    configuracion_complementos = {}  # Diccionario vacío en caso de error

=== CARGANDO CONFIGURACIÓN DE COMPLEMENTOS ===
Cargando configuración de COMPLEMENTOS SALARIALES...
✓ Configuración de complementos salariales cargada: 101 códigos

Cargando configuración de COMPLEMENTOS EXTRASALARIALES...
✓ Configuración de complementos extrasalariales cargada: 27 códigos

=== RESUMEN DE CONFIGURACIÓN DE COMPLEMENTOS ===
Total de códigos configurados: 128

Estadísticas:
  - Complementos salariales: 101
  - Complementos extrasalariales: 27
  - Normalizables: 11
  - Anualizables: 11


In [73]:
# DEFINICIÓN DE FUNCIONES DE EQUIPARACIÓN
print("=== DEFINIENDO FUNCIONES DE EQUIPARACIÓN ===")

def calcular_coef_tp(valor_coef_tp):
    """
    Convierte el coeficiente de tiempo parcial a decimal.
    Si está en porcentaje (>1), lo divide por 100.
    """
    if pd.isna(valor_coef_tp):
        return 1.0
    
    # Si el valor es mayor que 1, asumimos que está en porcentaje
    if valor_coef_tp > 1:
        return valor_coef_tp / 100
    
    return valor_coef_tp

def equiparar_salario_base(salario_base_efectivo, coef_tp, meses_trabajados):
    """
    Equipara el salario base aplicando factores de normalización y anualización.
    """
    if pd.isna(salario_base_efectivo) or salario_base_efectivo == 0:
        return 0
    
    # Aplicar normalización (jornada completa)
    if pd.isna(coef_tp) or coef_tp == 0:
        coef_tp = 1.0
    salario_normalizado = salario_base_efectivo * (1/coef_tp)
    
    # Aplicar anualización (12 meses)
    if pd.isna(meses_trabajados) or meses_trabajados == 0:
        meses_trabajados = 12
    salario_equiparado = salario_normalizado * (12/meses_trabajados)
    
    return salario_equiparado

def equiparar_complemento(complemento_efectivo, coef_tp, meses_trabajados, es_normalizable, es_anualizable):
    """
    Equipara un complemento aplicando factores según su configuración.
    Si no es ni normalizable ni anualizable, no se procesa.
    """
    if pd.isna(complemento_efectivo) or complemento_efectivo == 0:
        return 0
    
    # Si no es ni normalizable ni anualizable, no procesamos este complemento
    if not es_normalizable and not es_anualizable:
        return complemento_efectivo  # Retornamos el valor original sin procesar
    
    resultado = complemento_efectivo
    
    # Aplicar normalización si es normalizable
    if es_normalizable:
        if pd.isna(coef_tp) or coef_tp == 0:
            coef_tp = 1.0
        resultado = resultado * (1/coef_tp)
    
    # Aplicar anualización si es anualizable
    if es_anualizable:
        if pd.isna(meses_trabajados) or meses_trabajados == 0:
            meses_trabajados = 12
        resultado = resultado * (12/meses_trabajados)
    
    return resultado

def obtener_config_complemento(codigo_complemento, configuracion_complementos):
    """
    Obtiene la configuración de un complemento específico.
    Retorna tupla (es_normalizable, es_anualizable, tipo)
    """
    # Intentar búsqueda directa primero
    if codigo_complemento in configuracion_complementos:
        config = configuracion_complementos[codigo_complemento]
        return config['es_normalizable'], config['es_anualizable'], config['tipo']
    
    # Si no se encuentra, intentar búsquedas alternativas
    # Para códigos como '45', buscar 'PS45'
    if codigo_complemento.isdigit():
        codigo_ps = f"PS{codigo_complemento}"
        if codigo_ps in configuracion_complementos:
            config = configuracion_complementos[codigo_ps]
            return config['es_normalizable'], config['es_anualizable'], config['tipo']
    
    # Para códigos como 'PS45', buscar también '45'
    if codigo_complemento.startswith('PS') and codigo_complemento[2:].isdigit():
        codigo_num = codigo_complemento[2:]
        if codigo_num in configuracion_complementos:
            config = configuracion_complementos[codigo_num]
            return config['es_normalizable'], config['es_anualizable'], config['tipo']
    
    # Valores por defecto si no se encuentra la configuración
    print(f"⚠️ Configuración no encontrada para {codigo_complemento}, marcando como no procesable")
    return False, False, 'desconocido'  # Por defecto: NO normalizable y NO anualizable

def equiparar_complemento_individual(valor_efectivo, coef_tp, meses_trabajados, codigo_ps, configuracion_complementos):
    """
    Equipara un complemento individual usando su configuración específica.
    """
    if pd.isna(valor_efectivo) or valor_efectivo == 0:
        return 0
    
    # Obtener configuración específica del complemento
    es_normalizable, es_anualizable, tipo = obtener_config_complemento(codigo_ps, configuracion_complementos)
    
    # Aplicar equiparación
    return equiparar_complemento(valor_efectivo, coef_tp, meses_trabajados, es_normalizable, es_anualizable)

print("✓ Funciones de equiparación definidas correctamente")

=== DEFINIENDO FUNCIONES DE EQUIPARACIÓN ===
✓ Funciones de equiparación definidas correctamente


In [74]:
print("=== CALCULANDO VARIABLES EQUIPARADAS ===")

# Crear df_equiparado si no existe
if 'df_equiparado' not in locals():
    df_equiparado = df.copy()
    print("✓ df_equiparado creado desde df original")

try:
    # Verificar que tenemos las columnas necesarias
    columnas_criticas = ['meses_trabajados', 'coef_tp', 'salario_base_efectivo']
    if all(col in mapeo_columnas for col in columnas_criticas):
        
        # Obtener las columnas
        col_meses = mapeo_columnas['meses_trabajados']
        col_coef_tp = mapeo_columnas['coef_tp']
        col_sb_efectivo = mapeo_columnas['salario_base_efectivo']
        
        print(f"Usando columnas principales:")
        print(f"  - Meses trabajados: {col_meses}")
        print(f"  - % de jornada: {col_coef_tp}")
        print(f"  - SB efectivo: {col_sb_efectivo}")
        
        # Calcular coef_tp correctamente desde la columna "% de jornada"
        # Aplicar la función calcular_coef_tp para convertir porcentajes a decimales si es necesario
        df_equiparado['coef_tp_calculado'] = df_equiparado[col_coef_tp].apply(calcular_coef_tp)
        
        # Calcular salario base equiparado
        df_equiparado['salario_base_equiparado'] = df_equiparado.apply(
            lambda row: equiparar_salario_base(
                row[col_sb_efectivo], 
                row['coef_tp_calculado'], 
                row[col_meses]
            ), axis=1
        )
        
        print(f"\n✓ Salario base equiparado calculado")
        print(f"  - Valores únicos de coef_tp: {sorted(df_equiparado['coef_tp_calculado'].unique())}")
        print(f"  - SB efectivo promedio: {df_equiparado[col_sb_efectivo].mean():.2f}")
        print(f"  - SB equiparado promedio: {df_equiparado['salario_base_equiparado'].mean():.2f}")
        
        # Calcular complementos equiparados si existen
        if 'complementos_salariales_efectivo' in mapeo_columnas:
            col_cs_efectivo = mapeo_columnas['complementos_salariales_efectivo']
            # NOTA: Este cálculo se hará correctamente más adelante sumando individualmente
            # los complementos procesables vs no procesables
            print(f"  - Columna CS efectivo identificada: {col_cs_efectivo}")
        
        if 'complementos_extrasalariales_efectivo' in mapeo_columnas:
            col_ce_efectivo = mapeo_columnas['complementos_extrasalariales_efectivo']
            # NOTA: Este cálculo se hará correctamente más adelante sumando individualmente
            # los complementos procesables vs no procesables
            print(f"  - Columna CE efectivo identificada: {col_ce_efectivo}")
        
        # Procesar complementos PS y PE individuales si hay configuración
        if 'configuracion_complementos' in locals() and configuracion_complementos:
            print(f"\n=== PROCESANDO COMPLEMENTOS INDIVIDUALES ===")
            complementos_procesados = 0
            
            # Buscar todas las columnas PS (salariales) en el DataFrame
            columnas_ps = [col for col in df_equiparado.columns if col.startswith('PS') and col[2:].isdigit() and not col.endswith('_equiparado')]
            columnas_pe = [col for col in df_equiparado.columns if col.startswith('PE') and col[2:].isdigit() and not col.endswith('_equiparado')]
            
            print(f"Columnas PS encontradas: {columnas_ps}")
            print(f"Columnas PE encontradas: {columnas_pe}")
            
            # Procesar complementos PS (salariales)
            for col_ps in columnas_ps:
                codigo_ps = col_ps  # PS45, PS1, etc.
                
                # Obtener configuración del complemento
                es_normalizable, es_anualizable, tipo = obtener_config_complemento(codigo_ps, configuracion_complementos)
                
                # Solo procesar si es al menos normalizable O anualizable
                if es_normalizable or es_anualizable:
                    col_equiparado = f"{codigo_ps}_equiparado"
                    
                    # Verificar si hay datos en esta columna
                    datos_no_nulos = df_equiparado[col_ps].dropna()
                    if len(datos_no_nulos) > 0:
                        print(f"  Procesando {codigo_ps}: {len(datos_no_nulos)} registros con datos (N:{es_normalizable}, A:{es_anualizable})")
                        
                        # Equiparar usando configuración específica
                        df_equiparado[col_equiparado] = df_equiparado.apply(
                            lambda row: equiparar_complemento_individual(
                                row[col_ps], 
                                row['coef_tp_calculado'], 
                                row[mapeo_columnas['meses_trabajados']],
                                codigo_ps,
                                configuracion_complementos
                            ), axis=1
                        )
                        complementos_procesados += 1
                        
                        promedio_efectivo = df_equiparado[col_ps].mean()
                        promedio_equiparado = df_equiparado[col_equiparado].mean()
                        print(f"    Efectivo promedio: {promedio_efectivo:.2f}, Equiparado promedio: {promedio_equiparado:.2f}")
                else:
                    print(f"  Saltando {codigo_ps}: No es normalizable ni anualizable")
            
            # Procesar complementos PE (extrasalariales)
            for col_pe in columnas_pe:
                codigo_pe = col_pe  # PE1, PE2, etc.
                
                # Obtener configuración del complemento
                es_normalizable, es_anualizable, tipo = obtener_config_complemento(codigo_pe, configuracion_complementos)
                
                # Solo procesar si es al menos normalizable O anualizable
                if es_normalizable or es_anualizable:
                    col_equiparado = f"{codigo_pe}_equiparado"
                    
                    # Verificar si hay datos en esta columna
                    datos_no_nulos = df_equiparado[col_pe].dropna()
                    if len(datos_no_nulos) > 0:
                        print(f"  Procesando {codigo_pe}: {len(datos_no_nulos)} registros con datos (N:{es_normalizable}, A:{es_anualizable})")
                        
                        # Equiparar usando configuración específica
                        df_equiparado[col_equiparado] = df_equiparado.apply(
                            lambda row: equiparar_complemento_individual(
                                row[col_pe], 
                                row['coef_tp_calculado'], 
                                row[mapeo_columnas['meses_trabajados']],
                                codigo_pe,
                                configuracion_complementos
                            ), axis=1
                        )
                        complementos_procesados += 1
                        
                        promedio_efectivo = df_equiparado[col_pe].mean()
                        promedio_equiparado = df_equiparado[col_equiparado].mean()
                        print(f"    Efectivo promedio: {promedio_efectivo:.2f}, Equiparado promedio: {promedio_equiparado:.2f}")
                else:
                    print(f"  Saltando {codigo_pe}: No es normalizable ni anualizable")
            
            print(f"\n✓ Complementos individuales procesados: {complementos_procesados}")
            
            # CALCULAR TOTALES CORRECTOS DE COMPLEMENTOS EQUIPARADOS
            print(f"\n=== CALCULANDO TOTALES CORRECTOS ===")
            
            def calcular_total_complementos_equiparados_interno(row, tipo_complemento='PS'):
                """Calcula el total correcto: equiparado si es procesable, efectivo si no lo es"""
                total = 0
                
                if tipo_complemento == 'PS':
                    columnas_originales = [col for col in df_equiparado.columns if col.startswith('PS') and col[2:].isdigit() and not col.endswith('_equiparado')]
                else:  # PE
                    columnas_originales = [col for col in df_equiparado.columns if col.startswith('PE') and col[2:].isdigit() and not col.endswith('_equiparado')]
                
                for col_original in columnas_originales:
                    valor_original = row[col_original]
                    
                    if pd.notna(valor_original) and valor_original != 0:
                        col_equiparada = f"{col_original}_equiparado"
                        
                        if col_equiparada in df_equiparado.columns:
                            # Existe columna equiparada, usar ese valor (complemento procesable)
                            valor_equiparado = row[col_equiparada]
                            if pd.notna(valor_equiparado):
                                total += valor_equiparado
                            else:
                                total += valor_original
                        else:
                            # No existe columna equiparada, complemento no procesable, usar original
                            total += valor_original
                
                return total
            
            # Calcular totales correctos
            df_equiparado['complementos_salariales_equiparados'] = df_equiparado.apply(
                lambda row: calcular_total_complementos_equiparados_interno(row, 'PS'), axis=1
            )
            
            df_equiparado['complementos_extrasalariales_equiparados'] = df_equiparado.apply(
                lambda row: calcular_total_complementos_equiparados_interno(row, 'PE'), axis=1
            )
            
            print(f"  - CS equiparados promedio: {df_equiparado['complementos_salariales_equiparados'].mean():.2f}")
            print(f"  - CE equiparados promedio: {df_equiparado['complementos_extrasalariales_equiparados'].mean():.2f}")
        else:
            print(f"\n⚠️  No hay configuración de complementos disponible")
        
        print(f"\n✓ Variables equiparadas calculadas correctamente")
        
    else:
        print("❌ No se encontraron todas las columnas críticas necesarias")
        columnas_faltantes = [col for col in columnas_criticas if col not in mapeo_columnas]
        print(f"Columnas faltantes: {columnas_faltantes}")

except Exception as e:
    print(f"❌ Error en el cálculo de variables equiparadas: {str(e)}")
    import traceback
    traceback.print_exc()

=== CALCULANDO VARIABLES EQUIPARADAS ===
Usando columnas principales:
  - Meses trabajados: ¿Cuántos meses ha trabajado?
  - % de jornada: % de jornada
  - SB efectivo: Salario base anual efectivo

✓ Salario base equiparado calculado
  - Valores únicos de coef_tp: [np.float64(0.33), np.float64(0.5), np.float64(0.75), np.float64(0.758), np.float64(0.7802), np.float64(0.875), np.float64(1.0)]
  - SB efectivo promedio: 18916.67
  - SB equiparado promedio: 22309.58
  - Columna CS efectivo identificada: Compltos Salariales efectivo
  - Columna CE efectivo identificada: Compltos Extrasalariales efectivo

=== PROCESANDO COMPLEMENTOS INDIVIDUALES ===
Columnas PS encontradas: ['PS1', 'PS2', 'PS3', 'PS4', 'PS5', 'PS6', 'PS7', 'PS8', 'PS9', 'PS10', 'PS11', 'PS12', 'PS13', 'PS14', 'PS15', 'PS16', 'PS17', 'PS18', 'PS19', 'PS20', 'PS21', 'PS22', 'PS23', 'PS24', 'PS25', 'PS26', 'PS27', 'PS28', 'PS29', 'PS30', 'PS31', 'PS32', 'PS33', 'PS34', 'PS35', 'PS36', 'PS37', 'PS38', 'PS39', 'PS40', 'PS41', 'PS4

In [75]:
# VERIFICACIÓN Y VISUALIZACIÓN DE RESULTADOS EQUIPARADOS
if 'df_equiparado' in locals():
    print("=== VERIFICACIÓN DE DATOS EQUIPARADOS ===")
    
    # Mostrar estadísticas comparativas
    columnas_comparacion = []
    
    # Comparar salario base
    if 'salario_base_equiparado' in df_equiparado.columns:
        col_sb_original = mapeo_columnas.get('salario_base_efectivo')
        if col_sb_original:
            print(f"\n--- SALARIO BASE ---")
            print(f"Efectivo promedio: {df_equiparado[col_sb_original].mean():.2f}")
            print(f"Equiparado promedio: {df_equiparado['salario_base_equiparado'].mean():.2f}")
            print(f"Diferencia: {df_equiparado['salario_base_equiparado'].mean() - df_equiparado[col_sb_original].mean():.2f}")
            columnas_comparacion.extend([col_sb_original, 'salario_base_equiparado'])
    
    # Comparar totales si existen
    if 'sb_mas_comp_total_equiparado' in df_equiparado.columns:
        print(f"\n--- SALARIO BASE + COMPLEMENTOS TOTAL ---")
        print(f"Equiparado promedio: {df_equiparado['sb_mas_comp_total_equiparado'].mean():.2f}")
        columnas_comparacion.append('sb_mas_comp_total_equiparado')
    
    # Mostrar las primeras filas de las columnas clave
    if columnas_comparacion:
        print(f"\n=== PRIMERAS 5 FILAS - COMPARACIÓN ===")
        display(df_equiparado[columnas_comparacion].head())
    
    # Mostrar estadísticas básicas de equiparación (los cálculos correctos están en celdas posteriores)
    if 'salario_base_equiparado' in df_equiparado.columns:
        print(f"\n=== ESTADÍSTICAS BÁSICAS DE EQUIPARACIÓN ===")
        print(f"Total registros en DataFrame: {len(df_equiparado)}")
        print(f"Variables equiparadas creadas: {len([col for col in df_equiparado.columns if 'equiparado' in col])}")
        print(f"SB equiparado promedio (todos los registros): {df_equiparado['salario_base_equiparado'].mean():.2f}")
        print(f"⚠️  NOTA: Los cálculos finales según las reglas específicas se realizan en las celdas siguientes")
    
    # Resumen final
    print(f"\n=== RESUMEN FINAL ===")
    print(f"✓ DataFrame equiparado creado: {df_equiparado.shape[0]} filas x {df_equiparado.shape[1]} columnas")
    print(f"✓ Variables equiparadas disponibles para análisis")
    print(f"✓ Listo para cálculos de promedios y medianas")
    
else:
    print("❌ No se han calculado los datos equiparados. Ejecuta la celda anterior primero.")

=== VERIFICACIÓN DE DATOS EQUIPARADOS ===

--- SALARIO BASE ---
Efectivo promedio: 18916.67
Equiparado promedio: 22309.58
Diferencia: 3392.91

=== PRIMERAS 5 FILAS - COMPARACIÓN ===


,Salario base anual efectivo,salario_base_equiparado
0,1077.74,24585.94375
1,12443.63,12443.63000
2,41892.48,41892.48000
3,6618.22,6618.22000
4,27190.66,27190.66000



=== ESTADÍSTICAS BÁSICAS DE EQUIPARACIÓN ===
Total registros en DataFrame: 150
Variables equiparadas creadas: 13
SB equiparado promedio (todos los registros): 22309.58
⚠️  NOTA: Los cálculos finales según las reglas específicas se realizan en las celdas siguientes

=== RESUMEN FINAL ===
✓ DataFrame equiparado creado: 150 filas x 280 columnas
✓ Variables equiparadas disponibles para análisis
✓ Listo para cálculos de promedios y medianas


In [76]:
# IDENTIFICACIÓN DE REGISTROS "EX" Y APLICACIÓN DE REGLAS DE CÁLCULO
print("=== IDENTIFICACIÓN DE REGISTROS EX Y APLICACIÓN DE REGLAS ===")

# Identificar registros "Ex" en la columna "Reg."
col_reg = 'Reg.'
if col_reg in df_equiparado.columns:
    print(f"✓ Columna '{col_reg}' encontrada")
    print(f"Valores únicos en la columna: {df_equiparado[col_reg].dropna().unique()}")
    
    # Identificar registros "Ex"
    registros_ex = df_equiparado[col_reg].astype(str).str.contains('Ex', case=True, na=False)
    print(f"Registros 'Ex' encontrados: {registros_ex.sum()}")
    print(f"Total registros: {len(df_equiparado)}")
    print(f"Registros activos (no Ex): {(~registros_ex).sum()}")
    
    # Agregar información de registros ex al DataFrame equiparado
    df_equiparado['es_registro_ex'] = registros_ex
    
    # Filtrar registros activos (excluir "Ex")
    df_activos = df_equiparado[~df_equiparado['es_registro_ex']].copy()
    print(f"\n✓ DataFrame filtrado creado: {len(df_activos)} registros activos")
    
else:
    print(f"❌ Columna '{col_reg}' no encontrada")
    print("⚠️  Asumiendo que no hay registros Ex (todos los registros son activos)")
    df_equiparado['es_registro_ex'] = False
    df_activos = df_equiparado.copy()
    registros_ex = pd.Series([False] * len(df_equiparado))
    print(f"✓ DataFrame filtrado creado: {len(df_activos)} registros activos")

print(f"\n=== VERIFICACIÓN DE COLUMNAS PARA CÁLCULOS ===")

# Verificar que tenemos las columnas necesarias para los cálculos
columnas_efectivo_total = {
    'sb_efectivo_total': 'Salario base efectivo Total',
    'comp_salariales_efectivo_total': 'Compltos Salariales efectivo Total ',
    'comp_extrasalariales_efectivo_total': 'Compltos Extrasalariales efectivo Total '
}

# Verificar columnas efectivo total
print("Columnas EFECTIVO TOTAL (para datos efectivos):")
for clave, nombre in columnas_efectivo_total.items():
    if nombre in df_activos.columns:
        print(f"✓ {clave}: {nombre}")
    else:
        print(f"❌ {clave}: {nombre} - NO ENCONTRADA")

=== IDENTIFICACIÓN DE REGISTROS EX Y APLICACIÓN DE REGLAS ===
✓ Columna 'Reg.' encontrada
Valores únicos en la columna: [' ' 'Ex']
Registros 'Ex' encontrados: 6
Total registros: 150
Registros activos (no Ex): 144

✓ DataFrame filtrado creado: 144 registros activos

=== VERIFICACIÓN DE COLUMNAS PARA CÁLCULOS ===
Columnas EFECTIVO TOTAL (para datos efectivos):
✓ sb_efectivo_total: Salario base efectivo Total
✓ comp_salariales_efectivo_total: Compltos Salariales efectivo Total 
✓ comp_extrasalariales_efectivo_total: Compltos Extrasalariales efectivo Total 


In [77]:
# CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EFECTIVOS
print("=== CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EFECTIVOS ===")

# Verificar que tenemos df_activos
if 'df_activos' not in locals():
    df_activos = df_equiparado[~df_equiparado.get('es_registro_ex', False)].copy()

# Definir columnas para análisis
col_sb_efectivo = 'Salario base anual efectivo'
col_comp_sal_efectivo = 'Compltos Salariales efectivo'
col_comp_extra_efectivo = 'Compltos Extrasalariales efectivo'

# SALARIO BASE EFECTIVO
if col_sb_efectivo in df_activos.columns:
    print(f"\n--- SALARIO BASE EFECTIVO ---")
    personas_sb_efectivo_positivo = df_activos[df_activos[col_sb_efectivo] > 0]
    sb_efectivo_positivo = personas_sb_efectivo_positivo[col_sb_efectivo]
    
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Personas con SB > 0: {len(personas_sb_efectivo_positivo)}")
    print(f"Suma SB efectivos (SB > 0): {sb_efectivo_positivo.sum():,.2f}")
    print(f"Promedio SB efectivo: {sb_efectivo_positivo.mean():.2f}")
    print(f"Mediana SB efectivo (SB > 0): {sb_efectivo_positivo.median():.2f}")

# COMPLEMENTOS SALARIALES
if col_comp_sal_efectivo in df_activos.columns:
    # Crear columna combinada SB + Comp Salariales efectivo
    df_activos['sb_comp_sal_efectivo'] = df_activos[col_sb_efectivo] + df_activos[col_comp_sal_efectivo]
    
    print(f"\n--- SALARIO BASE + COMPLEMENTOS SALARIALES EFECTIVOS ---")
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Promedio SB + Comp Sal efectivo: {df_activos['sb_comp_sal_efectivo'].mean():,.2f}")
    print(f"Mediana SB + Comp Sal efectivo: {df_activos['sb_comp_sal_efectivo'].median():,.2f}")

# COMPLEMENTOS TOTALES
if col_comp_extra_efectivo in df_activos.columns:
    # Crear columna combinada SB + Comp Total efectivo
    df_activos['sb_comp_total_efectivo'] = (df_activos[col_sb_efectivo] + 
                                          df_activos[col_comp_sal_efectivo] + 
                                          df_activos[col_comp_extra_efectivo])
    
    print(f"\n--- SALARIO BASE + COMPLEMENTOS TOTALES EFECTIVOS ---")
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Promedio SB + Comp Total efectivo: {df_activos['sb_comp_total_efectivo'].mean():,.2f}")
    print(f"Mediana SB + Comp Total efectivo: {df_activos['sb_comp_total_efectivo'].median():,.2f}")

=== CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EFECTIVOS ===

--- SALARIO BASE EFECTIVO ---
Personas totales (sin ex): 144
Personas con SB > 0: 139
Suma SB efectivos (SB > 0): 2,693,166.80
Promedio SB efectivo: 19375.30
Mediana SB efectivo (SB > 0): 23083.43

--- SALARIO BASE + COMPLEMENTOS SALARIALES EFECTIVOS ---
Personas totales (sin ex): 144
Promedio SB + Comp Sal efectivo: 34,330.78
Mediana SB + Comp Sal efectivo: 35,595.78

--- SALARIO BASE + COMPLEMENTOS TOTALES EFECTIVOS ---
Personas totales (sin ex): 144
Promedio SB + Comp Total efectivo: 34,426.64
Mediana SB + Comp Total efectivo: 37,012.29


In [78]:
# CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EQUIPARADOS
print("=== CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EQUIPARADOS ===")

# SALARIO BASE EQUIPARADO
if 'salario_base_equiparado' in df_activos.columns:
    print(f"\n--- SALARIO BASE EQUIPARADO ---")
    personas_sb_efectivo_positivo = df_activos[df_activos[col_sb_efectivo] > 0]
    
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Personas con SB efectivo > 0: {len(personas_sb_efectivo_positivo)}")
    
    # Calcular sobre registros con SB efectivo > 0
    suma_sb_equiparados = personas_sb_efectivo_positivo['salario_base_equiparado'].sum()
    promedio_sb_equiparado = personas_sb_efectivo_positivo['salario_base_equiparado'].mean()
    mediana_sb_equiparado = personas_sb_efectivo_positivo['salario_base_equiparado'].median()
    
    print(f"Suma SB equiparados (donde SB efectivo > 0): {suma_sb_equiparados:,.2f}")
    print(f"Promedio SB equiparado: {promedio_sb_equiparado:.2f}")
    print(f"Mediana SB equiparado (SB efectivo > 0): {mediana_sb_equiparado:.2f}")

# COMPLEMENTOS SALARIALES EQUIPARADOS
if 'complementos_salariales_equiparados' in df_activos.columns:
    # Crear columna combinada SB + Comp Salariales equiparado
    df_activos['sb_comp_sal_equiparado'] = df_activos['salario_base_equiparado'] + df_activos['complementos_salariales_equiparados']
    
    print(f"\n--- SALARIO BASE + COMPLEMENTOS SALARIALES EQUIPARADOS ---")
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Promedio SB + Comp Sal equiparado: {df_activos['sb_comp_sal_equiparado'].mean():,.2f}")
    print(f"Mediana SB + Comp Sal equiparado: {df_activos['sb_comp_sal_equiparado'].median():,.2f}")

# COMPLEMENTOS TOTALES EQUIPARADOS
if 'complementos_extrasalariales_equiparados' in df_activos.columns:
    # Crear columna combinada SB + Comp Total equiparado
    df_activos['sb_comp_total_equiparado'] = (df_activos['salario_base_equiparado'] + 
                                            df_activos.get('complementos_salariales_equiparados', 0) + 
                                            df_activos['complementos_extrasalariales_equiparados'])
    
    print(f"\n--- SALARIO BASE + COMPLEMENTOS TOTALES EQUIPARADOS ---")
    print(f"Personas totales (sin ex): {len(df_activos)}")
    print(f"Promedio SB + Comp Total equiparado: {df_activos['sb_comp_total_equiparado'].mean():,.2f}")
    print(f"Mediana SB + Comp Total equiparado: {df_activos['sb_comp_total_equiparado'].median():,.2f}")

=== CÁLCULOS DE PROMEDIOS Y MEDIANAS - DATOS EQUIPARADOS ===

--- SALARIO BASE EQUIPARADO ---
Personas totales (sin ex): 144
Personas con SB efectivo > 0: 139
Suma SB equiparados (donde SB efectivo > 0): 3,203,488.92
Promedio SB equiparado: 23046.68
Mediana SB equiparado (SB efectivo > 0): 23723.73

--- SALARIO BASE + COMPLEMENTOS SALARIALES EQUIPARADOS ---
Personas totales (sin ex): 144
Promedio SB + Comp Sal equiparado: 38,507.68
Mediana SB + Comp Sal equiparado: 36,093.85

--- SALARIO BASE + COMPLEMENTOS TOTALES EQUIPARADOS ---
Personas totales (sin ex): 144
Promedio SB + Comp Total equiparado: 38,541.66
Mediana SB + Comp Total equiparado: 36,143.85


In [79]:
# ANÁLISIS DE BRECHAS SALARIALES POR GÉNERO
print("=== ANÁLISIS DE BRECHAS SALARIALES POR GÉNERO ===")

# Verificar que tenemos la columna de sexo
col_sexo = 'Sexo'
if col_sexo in df_activos.columns:
    print(f"✓ Columna '{col_sexo}' encontrada")
    
    # Separar por género
    df_hombres_efectivo = df_activos[df_activos[col_sexo] == 'Hombres'].copy()
    df_mujeres_efectivo = df_activos[df_activos[col_sexo] == 'Mujeres'].copy()
    
    df_hombres_equiparado = df_hombres_efectivo.copy()
    df_mujeres_equiparado = df_mujeres_efectivo.copy()
    
    print(f"  - Hombres: {len(df_hombres_efectivo)}")
    print(f"  - Mujeres: {len(df_mujeres_efectivo)}")
    
    # Crear DataFrame de resumen de brechas
    metricas_brecha = {}
    
    # SALARIO BASE
    if 'salario_base_equiparado' in df_activos.columns:
        print(f"\n--- BRECHA SALARIAL BASE ---")
        
        # Datos efectivos
        promedio_sb_efectivo_hombres = df_hombres_efectivo[col_sb_efectivo].mean()
        promedio_sb_efectivo_mujeres = df_mujeres_efectivo[col_sb_efectivo].mean()
        brecha_sb_efectivo = ((promedio_sb_efectivo_hombres - promedio_sb_efectivo_mujeres) / promedio_sb_efectivo_hombres) * 100
        
        # Datos equiparados
        promedio_sb_equiparado_hombres = df_hombres_equiparado['salario_base_equiparado'].mean()
        promedio_sb_equiparado_mujeres = df_mujeres_equiparado['salario_base_equiparado'].mean()
        brecha_sb_equiparado = ((promedio_sb_equiparado_hombres - promedio_sb_equiparado_mujeres) / promedio_sb_equiparado_hombres) * 100
        
        print(f"Promedio Hombres (efectivo): {promedio_sb_efectivo_hombres:,.2f}")
        print(f"Promedio Mujeres (efectivo): {promedio_sb_efectivo_mujeres:,.2f}")
        print(f"Brecha efectiva: {brecha_sb_efectivo:.2f}%")
        print(f"Promedio Hombres (equiparado): {promedio_sb_equiparado_hombres:,.2f}")
        print(f"Promedio Mujeres (equiparado): {promedio_sb_equiparado_mujeres:,.2f}")
        print(f"Brecha equiparada: {brecha_sb_equiparado:.2f}%")
        
        # Guardar métricas
        metricas_brecha['salario_base'] = {
            'efectivo': {'hombres': promedio_sb_efectivo_hombres, 'mujeres': promedio_sb_efectivo_mujeres, 'brecha': brecha_sb_efectivo},
            'equiparado': {'hombres': promedio_sb_equiparado_hombres, 'mujeres': promedio_sb_equiparado_mujeres, 'brecha': brecha_sb_equiparado}
        }
    
    # Crear DataFrame de resumen
    df_resumen_brecha = pd.DataFrame({
        'Métrica': ['Salario Base'],
        'Hombres_Efectivo': [promedio_sb_efectivo_hombres],
        'Mujeres_Efectivo': [promedio_sb_efectivo_mujeres],
        'Brecha_Efectiva': [brecha_sb_efectivo],
        'Hombres_Equiparado': [promedio_sb_equiparado_hombres],
        'Mujeres_Equiparado': [promedio_sb_equiparado_mujeres],
        'Brecha_Equiparada': [brecha_sb_equiparado]
    })
    
    print(f"\n=== RESUMEN DE BRECHAS ===")
    display(df_resumen_brecha)
    
else:
    print(f"❌ Columna '{col_sexo}' no encontrada")

=== ANÁLISIS DE BRECHAS SALARIALES POR GÉNERO ===
✓ Columna 'Sexo' encontrada
  - Hombres: 102
  - Mujeres: 42

--- BRECHA SALARIAL BASE ---
Promedio Hombres (efectivo): 19,552.72
Promedio Mujeres (efectivo): 18,951.23
Brecha efectiva: 3.08%
Promedio Hombres (equiparado): 22,492.89
Promedio Mujeres (equiparado): 21,647.96
Brecha equiparada: 3.76%

=== RESUMEN DE BRECHAS ===


,Métrica,Hombres_Efectivo,Mujeres_Efectivo,Brecha_Efectiva,Hombres_Equiparado,Mujeres_Equiparado,Brecha_Equiparada
0,Salario Base,19552.717041,18951.232439,3.07622,22492.88979,21647.956123,3.756448


In [80]:
# RESUMEN FINAL Y PREPARACIÓN PARA EXPORTACIÓN
print("=== RESUMEN FINAL Y PREPARACIÓN PARA EXPORTACIÓN ===")

# Crear DataFrame final para exportar (df_procesado)
df_procesado = df_equiparado.copy()

# Verificar integridad del DataFrame final
print(f"DataFrame final para exportar:")
print(f"  - Dimensiones: {df_procesado.shape[0]} filas x {df_procesado.shape[1]} columnas")
print(f"  - Registros activos: {len(df_procesado[~df_procesado.get('es_registro_ex', False)])}")
print(f"  - Variables equiparadas: {len([col for col in df_procesado.columns if 'equiparado' in col])}")

# Verificar específicamente PS45 equiparado
if 'PS45' in df_procesado.columns and 'PS45_equiparado' in df_procesado.columns:
    registros_ps45 = df_procesado[df_procesado['PS45'].notna() & (df_procesado['PS45'] > 0)]
    print(f"\n=== VERIFICACIÓN PS45 ===")
    print(f"Registros con PS45: {len(registros_ps45)}")
    
    if len(registros_ps45) > 0:
        # Buscar el registro orden 47 específicamente
        if 'Orden' in df_procesado.columns:
            orden_47 = df_procesado[df_procesado['Orden'] == 47]
            if len(orden_47) > 0:
                ps45_efectivo = orden_47['PS45'].iloc[0]
                ps45_equiparado = orden_47['PS45_equiparado'].iloc[0]
                print(f"Orden 47: PS45={ps45_efectivo:,.2f}, PS45_equiparado={ps45_equiparado:,.2f}")
                
                # Verificar si necesita corrección
                if pd.notna(ps45_efectivo) and ps45_efectivo > 0 and abs(ps45_equiparado - ps45_efectivo) < 1:
                    print("⚠️  PS45_equiparado parece no estar equiparado correctamente")
                    
                    # Aplicar corrección con configuración correcta
                    coef_tp = orden_47.get('coef_tp_calculado', orden_47.get('% de jornada', 1.0)).iloc[0]
                    meses = orden_47['¿Cuántos meses ha trabajado?'].iloc[0]
                    
                    if pd.notna(coef_tp) and pd.notna(meses) and coef_tp != 0 and meses != 0:
                        # Forzar configuración correcta para PS45
                        ps45_equiparado_correcto = ps45_efectivo * (1/coef_tp) * (12/meses)
                        df_procesado.loc[orden_47.index[0], 'PS45_equiparado'] = ps45_equiparado_correcto
                        print(f"✓ PS45_equiparado corregido a: {ps45_equiparado_correcto:,.2f}")

print(f"\n✓ DataFrame final preparado para exportación")

=== RESUMEN FINAL Y PREPARACIÓN PARA EXPORTACIÓN ===
DataFrame final para exportar:
  - Dimensiones: 150 filas x 281 columnas
  - Registros activos: 144
  - Variables equiparadas: 13

=== VERIFICACIÓN PS45 ===
Registros con PS45: 22
Orden 47: PS45=3,638.80, PS45_equiparado=24,407.26

✓ DataFrame final preparado para exportación


In [81]:
# GUARDAR DATOS PROCESADOS EN ARCHIVO EXCEL
print("=== GUARDANDO DATOS PROCESADOS ===")

# Definir la ruta del archivo de salida
nombre_base = os.path.splitext(os.path.basename(archivo_excel))[0]
ruta_procesado = os.path.join(os.path.dirname(archivo_excel), f"{nombre_base}_procesado.xlsx")

try:
    # Verificar que df_procesado existe
    if 'df_procesado' not in locals():
        df_procesado = df_equiparado.copy()
        print("⚠️  Usando df_equiparado como df_procesado")
    
    print(f"Guardando datos en: {ruta_procesado}")
    print(f"Dimensiones: {df_procesado.shape[0]} filas x {df_procesado.shape[1]} columnas")
    
    # Guardar el DataFrame procesado
    df_procesado.to_excel(ruta_procesado, index=False, engine='openpyxl')
    
    # Verificar que el archivo se guardó correctamente
    if os.path.exists(ruta_procesado):
        tamano_archivo = os.path.getsize(ruta_procesado) / (1024 * 1024)  # MB
        print(f"✓ Archivo guardado exitosamente")
        print(f"  - Ruta: {ruta_procesado}")
        print(f"  - Tamaño: {tamano_archivo:.2f} MB")
        
        # Verificación específica del valor PS45
        if 'PS45_equiparado' in df_procesado.columns and 'Orden' in df_procesado.columns:
            orden_47 = df_procesado[df_procesado['Orden'] == 47]
            if len(orden_47) > 0:
                ps45_eq_final = orden_47['PS45_equiparado'].iloc[0]
                print(f"  - PS45_equiparado para orden 47: {ps45_eq_final:,.2f}")
    else:
        print(f"❌ Error: El archivo no se pudo crear")
        
except Exception as e:
    print(f"❌ Error al guardar el archivo: {e}")
    import traceback
    traceback.print_exc()

print("\n=== PROCESO COMPLETADO ===")
print("✓ Análisis de datos completado")
print("✓ Variables equiparadas calculadas")
print("✓ Archivo procesado guardado")
print("\nPuedes revisar el archivo generado para ver todos los datos procesados.")

=== GUARDANDO DATOS PROCESADOS ===
Guardando datos en: C:\Users\juanf\Downloads\Equiality Momentum\datos\PRUEBA QLICK JULIO 2025_procesado.xlsx
Dimensiones: 150 filas x 281 columnas
✓ Archivo guardado exitosamente
  - Ruta: C:\Users\juanf\Downloads\Equiality Momentum\datos\PRUEBA QLICK JULIO 2025_procesado.xlsx
  - Tamaño: 0.14 MB
  - PS45_equiparado para orden 47: 24,407.26

=== PROCESO COMPLETADO ===
✓ Análisis de datos completado
✓ Variables equiparadas calculadas
✓ Archivo procesado guardado

Puedes revisar el archivo generado para ver todos los datos procesados.
✓ Archivo guardado exitosamente
  - Ruta: C:\Users\juanf\Downloads\Equiality Momentum\datos\PRUEBA QLICK JULIO 2025_procesado.xlsx
  - Tamaño: 0.14 MB
  - PS45_equiparado para orden 47: 24,407.26

=== PROCESO COMPLETADO ===
✓ Análisis de datos completado
✓ Variables equiparadas calculadas
✓ Archivo procesado guardado

Puedes revisar el archivo generado para ver todos los datos procesados.
